<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/transformer_prompting_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考：
1. https://xiaosheng.blog/2022/09/10/what-is-prompt
2. https://www.promptingguide.ai/zh/introduction/basics
3. https://github.com/f/awesome-chatgpt-prompts


# 数据集
中文情感分析语料库 ChnSentiCorp 作为数据集，其包含各类网络评论接近一万条；
语料已经划分好了训练集、验证集、测试集（分别包含 9600、1200、1200 条评论），一行是一个样本，使用 TAB 分隔评论和对应的标签，“0”表示消极，“1”表示积极。

tip:
NLP 大多是分类问题，标注的数据集开始大部分是人工，后续机器标注


In [5]:
!wget https://ernie-github.cdn.bcebos.com/data-chnsenticorp.tar.gz

--2023-12-12 04:00:31--  https://ernie-github.cdn.bcebos.com/data-chnsenticorp.tar.gz
Resolving ernie-github.cdn.bcebos.com (ernie-github.cdn.bcebos.com)... 182.106.158.35, 111.170.27.1, 113.219.142.35, ...
Connecting to ernie-github.cdn.bcebos.com (ernie-github.cdn.bcebos.com)|182.106.158.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1753724 (1.7M) [application/x-gzip]
Saving to: ‘data-chnsenticorp.tar.gz’

data-chnsenticorp.t 100%[===================>]   1.67M  1.19MB/s    in 1.4s    

2023-12-12 04:00:34 (1.19 MB/s) - ‘data-chnsenticorp.tar.gz’ saved [1753724/1753724]



In [6]:
!tar zxvf data-chnsenticorp.tar.gz

chnsenticorp/
chnsenticorp/dev/
chnsenticorp/dev/part.0
chnsenticorp/test/
chnsenticorp/test/part.0
chnsenticorp/train/
chnsenticorp/train/part.0


In [4]:
!ls -gh chnsenticorp/train/part.0
!head chnsenticorp/train/part.0
!ls -gh chnsenticorp/dev/part.0
!head chnsenticorp/dev/part.0
!ls -gh chnsenticorp/test/part.0
!head chnsenticorp/test/part.0


-rw-rw-r-- 1 501 2.9M Jul  2  2019 chnsenticorp/train/part.0
选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般	1
15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜欢数字小键盘，输数字特方便，样子也很美观，做工也相当不错	1
房间太小。其他的都一般。。。。。。。。。	0
1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来. 3.机盖的钢琴漆，手不能摸，一摸一个印. 4.硬盘分区不好办.	0
今天才知道这书还有第6卷,真有点郁闷:为什么同一套书有两种版本呢?当当网是不是该跟出版社商量商量,单独出个第6卷,让我们的孩子不会有所遗憾。	1
机器背面似乎被撕了张什么标签，残胶还在。但是又看不出是什么标签不见了，该有的都在，怪	0
呵呵，虽然表皮看上去不错很精致，但是我还是能看得出来是盗的。但是里面的内容真的不错，我妈爱看，我自己也学着找一些穴位。	0
这本书实在是太烂了,以前听浙大的老师说这本书怎么怎么不对,哪些地方都是误导的还不相信,终于买了一本看一下,发现真是~~~无语,这种书都写得出来	0
地理位置佳，在市中心。酒店服务好、早餐品种丰富。我住的商务数码房电脑宽带速度满意,房间还算干净，离湖南路小吃街近。	1
5.1期间在这住的，位置还可以，在市委市政府附近，要去商业区和步行街得打车，屋里有蚊子，虽然空间挺大，晚上熄灯后把窗帘拉上简直是伸手不见五指，很适合睡觉，但是会被该死的蚊子吵醒！打死了两只，第二天早上还是发现又没打死的，卫生间挺大，但是设备很老旧。	1
-rw-rw-r-- 1 501 366K Jul  2  2019 chnsenticorp/dev/part.0
這間酒店環境和服務態度亦算不錯,但房間空間太小~~不宣容納太大件行李~~且房間格調還可以~~ 中餐廳的廣東點心不太好吃~~要改善之~~~~但算價錢平宜~~可接受~~ 西餐廳格調都很好~~但吃的味道一般且令人等得太耐了~~要改善之~~	1
<荐书> 推荐所有喜欢<红楼>的红迷们一定要收藏这本书,要知道当年我听说这本书的时候花很长时间去图书馆找和借都没能如愿,所以

## 构造数据

In [5]:
from torch.utils.data import Dataset

class ChnSentiCorp(Dataset):
    def __init__(self, data_file):
        self.data = self.load_data(data_file)

    def load_data(self, data_file):
        Data = {}
        with open(data_file, 'rt', encoding='utf-8') as f:
            for idx, line in enumerate(f):
                items = line.strip().split('\t')
                assert len(items) == 2
                Data[idx] = {
                    'comment': items[0],
                    'label': items[1]
                }
        return Data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

train_data = ChnSentiCorp('chnsenticorp/train/part.0')
valid_data = ChnSentiCorp('chnsenticorp/dev/part.0')
test_data = ChnSentiCorp('chnsenticorp/test/part.0')


In [6]:
print(f'train set size: {len(train_data)}')
print(f'valid set size: {len(valid_data)}')
print(f'test set size: {len(test_data)}')
print(next(iter(train_data)))


train set size: 9600
valid set size: 1200
test set size: 1200
{'comment': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 'label': '1'}


最常见的 Prompting 方法就是借助模板将问题转换为 MLM 任务来解决。这里我们定义模板形式为“总体上来说很 [MASK]。x”，其中 x 表示评论文本，并且规定如果 [MASK]被预测为“好”就判定情感为“积极”，如果预测为“差”就判定为“消极”，即“积极”和“消极”标签对应的 label word 分别为“好 1”和“差 0”。



In [7]:
def get_prompt(x):
    prompt = f'总体上来说很[MASK]。{x}'
    return {
        'prompt': prompt,
        'mask_offset': prompt.find('[MASK]')
    }

def get_verbalizer(tokenizer):
    return {
        'pos': {'token': '好', 'id': tokenizer.convert_tokens_to_ids("好")},
        'neg': {'token': '差', 'id': tokenizer.convert_tokens_to_ids("差")}
    }


In [8]:
# BERT 分词器正确地将“[MASK]”识别为一个 token，并且记录下 [MASK] token 在序列中的索引
from transformers import AutoTokenizer

checkpoint = "bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

comment = '这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般。'

print('verbalizer:', get_verbalizer(tokenizer))

prompt_data = get_prompt(comment)
prompt, mask_offset = prompt_data['prompt'], prompt_data['mask_offset']

encoding = tokenizer(prompt, truncation=True)
tokens = encoding.tokens()
mask_idx = encoding.char_to_token(mask_offset)

print('prompt:', prompt)
print('prompt tokens:', tokens)
print('mask idx:', mask_idx)


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

verbalizer: {'pos': {'token': '好', 'id': 1962}, 'neg': {'token': '差', 'id': 2345}}
prompt: 总体上来说很[MASK]。这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般。
prompt tokens: ['[CLS]', '总', '体', '上', '来', '说', '很', '[MASK]', '。', '这', '个', '宾', '馆', '比', '较', '陈', '旧', '了', '，', '特', '价', '的', '房', '间', '也', '很', '一', '般', '。', '总', '体', '来', '说', '一', '般', '。', '[SEP]']
mask idx: 7


但是这种做法要求我们能够从词表中找到合适的 label word 来代表每一个类别，并且 label word 只能包含一个 token，而很多时候这是无法实现的。因此，另一种常见做法是为每个类别构建一个可学习的虚拟 token（又称伪 token），然后运用类别描述来初始化虚拟 token 的表示，最后使用这些虚拟 token 来扩展模型的 MLM 头。

例如，这里我们可以为“积极”和“消极”构建专门的虚拟 token “[POS]”和“[NEG]”，并且设置对应的类别描述为“好的、优秀的、正面的评价、积极的态度”和“差的、糟糕的、负面的评价、消极的态度”。下面我们扩展一下上面的 verbalizer 函数，添加一个 vtype 参数来区分两种 verbalizer 类型：

In [9]:
def get_verbalizer(tokenizer, vtype):
    assert vtype in ['base', 'virtual']
    return {
        'pos': {'token': '好', 'id': tokenizer.convert_tokens_to_ids("好")},
        'neg': {'token': '差', 'id': tokenizer.convert_tokens_to_ids("差")}
    } if vtype == 'base' else {
        'pos': {
            'token': '[POS]', 'id': tokenizer.convert_tokens_to_ids("[POS]"),
            'description': '好的、优秀的、正面的评价、积极的态度'
        },
        'neg': {
            'token': '[NEG]', 'id': tokenizer.convert_tokens_to_ids("[NEG]"),
            'description': '差的、糟糕的、负面的评价、消极的态度'
        }
    }

vtype = 'virtual'
# add label words
if vtype == 'virtual':
    tokenizer.add_special_tokens({'additional_special_tokens': ['[POS]', '[NEG]']})
print('verbalizer:', get_verbalizer(tokenizer, vtype=vtype))


verbalizer: {'pos': {'token': '[POS]', 'id': 21129, 'description': '好的、优秀的、正面的评价、积极的态度'}, 'neg': {'token': '[NEG]', 'id': 21128, 'description': '差的、糟糕的、负面的评价、消极的态度'}}


In [10]:
# Prompting 方法实际输入的是转换后的模板，而不是原始文本，因此我们首先使用模板函数 get_prompt() 来更新数据集：
class ChnSentiCorp(Dataset):
    def __init__(self, data_file):
        self.data = self.load_data(data_file)

    def load_data(self, data_file):
        Data = {}
        with open(data_file, 'rt', encoding='utf-8') as f:
            for idx, line in enumerate(f):
                items = line.strip().split('\t')
                assert len(items) == 2
                prompt_data = get_prompt(items[0])
                Data[idx] = {
                    'comment': items[0],
                    'prompt': prompt_data['prompt'],
                    'mask_offset': prompt_data['mask_offset'],
                    'label': items[1]
                }
        return Data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

train_data = ChnSentiCorp('chnsenticorp/train/part.0')
valid_data = ChnSentiCorp('chnsenticorp/dev/part.0')
test_data = ChnSentiCorp('chnsenticorp/test/part.0')

print(f'train set size: {len(train_data)}')
print(f'valid set size: {len(valid_data)}')
print(f'test set size: {len(test_data)}')
print(next(iter(train_data)))


train set size: 9600
valid set size: 1200
test set size: 1200
{'comment': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 'prompt': '总体上来说很[MASK]。选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 'mask_offset': 6, 'label': '1'}


## 数据预处理

In [11]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

vtype = 'base'
max_length = 512

checkpoint = "bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
if vtype == 'virtual':
    tokenizer.add_special_tokens({'additional_special_tokens': ['[POS]', '[NEG]']})

verbalizer = get_verbalizer(tokenizer, vtype='base')
pos_id, neg_id = verbalizer['pos']['id'], verbalizer['neg']['id']

def collate_fn(batch_samples):
    batch_sentences, batch_mask_idxs, batch_labels  = [], [], []
    for sample in batch_samples:
        batch_sentences.append(sample['prompt'])
        encoding = tokenizer(sample['prompt'], truncation=True)
        mask_idx = encoding.char_to_token(sample['mask_offset'])
        assert mask_idx is not None
        batch_mask_idxs.append(mask_idx)
        batch_labels.append(int(sample['label']))
    batch_inputs = tokenizer(
        batch_sentences,
        max_length=max_length,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    label_word_id = [neg_id, pos_id]
    return {
        'batch_inputs': batch_inputs,
        'batch_mask_idxs': batch_mask_idxs,
        'label_word_id': label_word_id,
        'labels': batch_labels
    }

train_dataloader = DataLoader(train_data, batch_size=4, shuffle=True, collate_fn=collate_fn)
valid_dataloader = DataLoader(valid_data, batch_size=4, shuffle=False, collate_fn=collate_fn)
test_dataloader = DataLoader(test_data, batch_size=4, shuffle=False, collate_fn=collate_fn)

batch_data = next(iter(train_dataloader))
print('batch_X shape:', {k: v.shape for k, v in batch_data['batch_inputs'].items()})
print(batch_data['batch_inputs'])
print(batch_data['batch_mask_idxs'])
print(batch_data['label_word_id'])
print(batch_data['labels'])


batch_X shape: {'input_ids': torch.Size([4, 73]), 'token_type_ids': torch.Size([4, 73]), 'attention_mask': torch.Size([4, 73])}
{'input_ids': tensor([[ 101, 2600,  860,  677, 3341, 6432, 2523,  103,  511, 8038, 2242, 2391,
         5682, 2506, 3300, 4157, 3909,  679, 7831, 5683, 3221, 1415, 5543, 6444,
          833, 4638, 3301, 1351, 7937, 4172, 1440, 6401, 2769,  671, 1898, 3336,
         3683, 7509, 1510,  671, 5663,  671, 4157,  856, 7509, 3766, 3300, 3300,
          856, 7509, 4152, 2218, 1962,  749,  102,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0],
        [ 101, 2600,  860,  677, 3341, 6432, 2523,  103,  511, 4685, 2496, 7770,
         4638, 2595,  817, 3683, 3353, 1213, 2972, 5773,  679, 5052,  794, 1525,
         3175, 7481, 3341, 6432, 4801,  816, 6963, 3766, 3300, 1922, 1920, 4638,
         7309, 7579, 2190,  754,  671, 5663, 4638,  782, 3341, 6432, 5318, 2190,
         1916, 4500,  749,  102,

# 训练模型

## 构建模型

In [12]:
import torch
from transformers import AutoModelForMaskedLM

checkpoint = "bert-base-chinese"
model = AutoModelForMaskedLM.from_pretrained(checkpoint)

text = "总体上来说很[MASK]。这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般。"
inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()


for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

# BERT 模型成功地将 [MASK] token 预测成了我们预期的表意词“好”。这里还打印出了其他几个大概率的预测词，大部分都具有积极的情感（“好”、“棒”、“赞”）

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


'>>> 总体上来说很好。这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般。'
'>>> 总体上来说很棒。这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般。'
'>>> 总体上来说很差。这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般。'
'>>> 总体上来说很般。这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般。'
'>>> 总体上来说很赞。这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般。'


In [13]:
# 直接使用bert模型不够灵活，基于bert重新构建模型BertForPrompt, 加入自定义的BertOnlyMLMHead进行推理
from torch import nn
from transformers.activations import ACT2FN
from transformers import AutoConfig
from transformers import BertPreTrainedModel, BertModel

def batched_index_select(input, dim, index):
    for i in range(1, len(input.shape)):
        if i != dim:
            index = index.unsqueeze(i)
    expanse = list(input.shape)
    expanse[0] = -1
    expanse[dim] = -1
    index = index.expand(expanse)
    return torch.gather(input, dim, index)

class BertPredictionHeadTransform(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        if isinstance(config.hidden_act, str):
            self.transform_act_fn = ACT2FN[config.hidden_act]
        else:
            self.transform_act_fn = config.hidden_act
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        hidden_states = self.dense(hidden_states)
        hidden_states = self.transform_act_fn(hidden_states)
        hidden_states = self.LayerNorm(hidden_states)
        return hidden_states

class BertLMPredictionHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.transform = BertPredictionHeadTransform(config)
        self.decoder = nn.Linear(config.hidden_size, config.vocab_size, bias=False)
        self.bias = nn.Parameter(torch.zeros(config.vocab_size))
        self.decoder.bias = self.bias

    def forward(self, hidden_states):
        hidden_states = self.transform(hidden_states)
        hidden_states = self.decoder(hidden_states)
        return hidden_states

class BertOnlyMLMHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.predictions = BertLMPredictionHead(config)

    def forward(self, sequence_output: torch.Tensor) -> torch.Tensor:
        prediction_scores = self.predictions(sequence_output)
        return prediction_scores

class BertForPrompt(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config, add_pooling_layer=False)
        self.cls = BertOnlyMLMHead(config)
        # Initialize weights and apply final processing
        self.post_init()

    def get_output_embeddings(self):
        return self.cls.predictions.decoder

    def set_output_embeddings(self, new_embeddings):
        self.cls.predictions.decoder = new_embeddings

    def forward(self, batch_inputs, batch_mask_idxs, label_word_id, labels=None):
        bert_output = self.bert(**batch_inputs)
        sequence_output = bert_output.last_hidden_state
        batch_mask_reps = batched_index_select(sequence_output, 1, batch_mask_idxs.unsqueeze(-1)).squeeze(1)
        prediction_scores = self.cls(batch_mask_reps)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(prediction_scores, labels)
        return loss, prediction_scores[:, label_word_id]

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')
checkpoint = "bert-base-chinese"
config = AutoConfig.from_pretrained(checkpoint)
model = BertForPrompt.from_pretrained(checkpoint, config=config).to(device)
if vtype == 'virtual':
    model.resize_token_embeddings(len(tokenizer))
    print(f"initialize embeddings of {verbalizer['pos']['token']} and {verbalizer['neg']['token']}")
    with torch.no_grad():
        pos_tokenized = tokenizer(verbalizer['pos']['description'])
        pos_tokenized_ids = tokenizer.convert_tokens_to_ids(pos_tokenized)
        neg_tokenized = tokenizer(verbalizer['neg']['description'])
        neg_tokenized_ids = tokenizer.convert_tokens_to_ids(neg_tokenized)
        new_embedding = model.bert.embeddings.word_embeddings.weight[pos_tokenized_ids].mean(axis=0)
        model.bert.embeddings.word_embeddings.weight[pos_id, :] = new_embedding.clone().detach().requires_grad_(True)
        new_embedding = model.bert.embeddings.word_embeddings.weight[neg_tokenized_ids].mean(axis=0)
        model.bert.embeddings.word_embeddings.weight[neg_id, :] = new_embedding.clone().detach().requires_grad_(True)

print(model)


Using cuda device
BertForPrompt(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-1

BERT 自带的 MLM head 由两个部分组成：首先对所有 token 进行一个 768x768 的非线性映射（包括激活函数和 LayerNorm），然后使用一个 768x21128 的线性映射预测词表中每个 token 的分数。



In [14]:
# 为了测试模型的操作是否符合预期，将一个 batch 的数据送入模型
def to_device(batch_data):
    new_batch_data = {}
    for k, v in batch_data.items():
        if k == 'batch_inputs':
            new_batch_data[k] = {
                k_: v_.to(device) for k_, v_ in v.items()
            }
        elif k == 'label_word_id':
            new_batch_data[k] = v
        else:
            new_batch_data[k] = torch.tensor(v).to(device)
    return new_batch_data

batch_data = next(iter(train_dataloader))
batch_data = to_device(batch_data)
_, outputs = model(**batch_data)
print(outputs.shape)

# 模型对每个样本都应该输出“消极”和“积极”两个类别对应 label word 的预测 logits 值，因此这里模型的输出尺寸 4x2 符合预期。

torch.Size([4, 2])


## 优化模型参数

每一轮 Epoch 分为“训练循环”和“验证/测试循环”，在训练循环中计算损失、优化模型参数，在验证/测试循环中评估模型性能

In [15]:
# 首先实现训练循环。
# 对标签词的预测实际上就是对类别的预测 损失是通过在类别预测和答案标签之间计算交叉熵：
from tqdm.auto import tqdm

def train_loop(dataloader, model, optimizer, lr_scheduler, epoch, total_loss):
    progress_bar = tqdm(range(len(dataloader)))
    progress_bar.set_description(f'loss: {0:>7f}')
    finish_step_num = epoch * len(dataloader)

    model.train()
    for step, batch_data in enumerate(dataloader, start=1):
        batch_data = to_device(batch_data)
        outputs = model(**batch_data)
        loss = outputs[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
        progress_bar.set_description(f'loss: {total_loss/(finish_step_num + step):>7f}')
        progress_bar.update(1)
    return total_loss


验证/测试循环负责评估模型。对于分类任务最常见的就是通过精确率、召回率、F1值 (P / R / F1) 指标来评估每个类别的预测性能，然后再通过宏/微 F1 值 (Macro-F1/Micro-F1) 来评估整体分类性能。



In [16]:
# 借助机器学习包 sklearn 提供的 classification_report 函数来输出这些指标
from sklearn.metrics import classification_report

y_true = [1, 1, 0, 1, 2, 1, 0, 2, 1, 1, 0, 1, 0]
y_pred = [1, 0, 0, 1, 2, 0, 1, 1, 1, 0, 0, 1, 0]

print(classification_report(y_true, y_pred, output_dict=False))


              precision    recall  f1-score   support

           0       0.50      0.75      0.60         4
           1       0.67      0.57      0.62         7
           2       1.00      0.50      0.67         2

    accuracy                           0.62        13
   macro avg       0.72      0.61      0.63        13
weighted avg       0.67      0.62      0.62        13



In [17]:
# 汇总模型对所有样本的预测结果和答案标签，然后送入到 classification_report 中计算各项分类指标
from sklearn.metrics import classification_report

def test_loop(dataloader, model):
    true_labels, predictions = [], []
    model.eval()
    with torch.no_grad():
        for batch_data in tqdm(dataloader):
            true_labels += batch_data['labels']
            batch_data = to_device(batch_data)
            outputs = model(**batch_data)
            pred = outputs[1]
            predictions += pred.argmax(dim=-1).cpu().numpy().tolist()
    metrics = classification_report(true_labels, predictions, output_dict=True)
    pos_p, pos_r, pos_f1 = metrics['1']['precision'], metrics['1']['recall'], metrics['1']['f1-score']
    neg_p, neg_r, neg_f1 = metrics['0']['precision'], metrics['0']['recall'], metrics['0']['f1-score']
    macro_f1, micro_f1 = metrics['macro avg']['f1-score'], metrics['weighted avg']['f1-score']
    print(f"pos: {pos_p*100:>0.2f} / {pos_r*100:>0.2f} / {pos_f1*100:>0.2f}, neg: {neg_p*100:>0.2f} / {neg_r*100:>0.2f} / {neg_f1*100:>0.2f}")
    print(f"Macro-F1: {macro_f1*100:>0.2f} Micro-F1: {micro_f1*100:>0.2f}\n")
    return metrics


## 保持模型

根据模型在验证集上的性能来调整超参数以及选出最好的模型权重，然后将选出的模型应用于测试集以评估最终的性能

In [18]:
# 在开始训练之前，我们先评估一下没有微调的 BERT 模型在测试集上的性能。

test_data = ChnSentiCorp('chnsenticorp/test/part.0')
test_dataloader = DataLoader(test_data, batch_size=4, shuffle=False, collate_fn=collate_fn)

test_loop(test_dataloader, model)


  0%|          | 0/300 [00:00<?, ?it/s]

pos: 53.05 / 100.00 / 69.33, neg: 100.00 / 9.12 / 16.72
Macro-F1: 43.02 Micro-F1: 43.37



{'0': {'precision': 1.0,
  'recall': 0.09121621621621621,
  'f1-score': 0.16718266253869968,
  'support': 592},
 '1': {'precision': 0.5305410122164049,
  'recall': 1.0,
  'f1-score': 0.69327251995439,
  'support': 608},
 'accuracy': 0.5516666666666666,
 'macro avg': {'precision': 0.7652705061082025,
  'recall': 0.5456081081081081,
  'f1-score': 0.43022759124654486,
  'support': 1200},
 'weighted avg': {'precision': 0.7621407795229784,
  'recall': 0.5516666666666666,
  'f1-score': 0.43373485696264946,
  'support': 1200}}

In [19]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5
epoch_num = 3
# 使用 AdamW 优化器，并且通过 get_scheduler() 函数定义学习率调度器
optimizer = AdamW(model.parameters(), lr=learning_rate)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_f1_score = 0.
for epoch in range(epoch_num):
    print(f"Epoch {epoch+1}/{epoch_num}\n" + 30 * "-")
    total_loss = train_loop(train_dataloader, model, optimizer, lr_scheduler, epoch, total_loss)
    valid_scores = test_loop(valid_dataloader, model)
    macro_f1, micro_f1 = valid_scores['macro avg']['f1-score'], valid_scores['weighted avg']['f1-score']
    f1_score = (macro_f1 + micro_f1) / 2
    if f1_score > best_f1_score:
        best_f1_score = f1_score
        print('saving new weights...\n')
        torch.save(
            model.state_dict(),
            f'epoch_{epoch+1}_valid_macrof1_{(macro_f1*100):0.3f}_microf1_{(micro_f1*100):0.3f}_model_weights.bin'
        )
print("Done!")


Epoch 1/3
------------------------------


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/2400 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

pos: 100.00 / 0.17 / 0.34, neg: 50.63 / 100.00 / 67.22
Macro-F1: 33.78 Micro-F1: 34.17

saving new weights...

Epoch 2/3
------------------------------


...

  0%|          | 0/300 [00:00<?, ?it/s]

pos: 100.00 / 2.02 / 3.97, neg: 51.09 / 100.00 / 67.63
Macro-F1: 35.80 Micro-F1: 36.17

saving new weights...

Epoch 3/3
------------------------------


  0%|          | 0/2400 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

pos: 98.98 / 16.36 / 28.08, neg: 54.99 / 99.84 / 70.92
Macro-F1: 49.50 Micro-F1: 49.75

saving new weights...

Done!


可以看到，得益于 Prompt 方法，不断微调的 BERT 模型也已经具有初步的情感分析能力，在测试集上的 Macro-F1 和 Micro-F1 值分别为 49.50 和 49.75。有趣的是，“积极”类别的召回率和“消极”类别的准确率都为 100%，这说明 BERT 对大部分样本都倾向于判断为“积极”类（可能预训练时看到的积极性文本更多吧）。

> tips:
如果采用虚拟 label word（设置 vtype = 'virtual' ），模型是无法直接进行预测的。在扩展了词表之后，MLM head 的参数矩阵尺寸也会进行调整，新加入的参数都是随机初始化的，此时必须进行微调才能让 MLM head 正常工作。



In [7]:
# 完整代码：
import os
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoConfig
from transformers import BertPreTrainedModel, BertModel
from transformers.activations import ACT2FN
from transformers import AdamW, get_scheduler
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

vtype = 'base'
max_length = 512
batch_size = 4
learning_rate = 1e-5
epoch_num = 3

def seed_everything(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

def get_prompt(x):
    prompt = f'总体上来说很[MASK]。{x}'
    return {
        'prompt': prompt,
        'mask_offset': prompt.find('[MASK]')
    }

def get_verbalizer(tokenizer, vtype):
    assert vtype in ['base', 'virtual']
    return {
        'pos': {'token': '好', 'id': tokenizer.convert_tokens_to_ids("好")},
        'neg': {'token': '差', 'id': tokenizer.convert_tokens_to_ids("差")}
    } if vtype == 'base' else {
        'pos': {
            'token': '[POS]', 'id': tokenizer.convert_tokens_to_ids("[POS]"),
            'description': '好的、优秀的、正面的评价、积极的态度'
        },
        'neg': {
            'token': '[NEG]', 'id': tokenizer.convert_tokens_to_ids("[NEG]"),
            'description': '差的、糟糕的、负面的评价、消极的态度'
        }
    }

seed_everything(12)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

class ChnSentiCorp(Dataset):
    def __init__(self, data_file):
        self.data = self.load_data(data_file)

    def load_data(self, data_file):
        Data = {}
        with open(data_file, 'rt', encoding='utf-8') as f:
            for idx, line in enumerate(f):
                items = line.strip().split('\t')
                assert len(items) == 2
                prompt_data = get_prompt(items[0])
                Data[idx] = {
                    'prompt': prompt_data['prompt'],
                    'mask_offset': prompt_data['mask_offset'],
                    'label': items[1]
                }
        return Data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

train_data = ChnSentiCorp('chnsenticorp/train/part.0')
valid_data = ChnSentiCorp('chnsenticorp/dev/part.0')
test_data = ChnSentiCorp('chnsenticorp/test/part.0')

checkpoint = "bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
if vtype == 'virtual':
    tokenizer.add_special_tokens({'additional_special_tokens': ['[POS]', '[NEG]']})

verbalizer = get_verbalizer(tokenizer, vtype=vtype)
pos_id, neg_id = verbalizer['pos']['id'], verbalizer['neg']['id']

def collate_fn(batch_samples):
    batch_sentences, batch_mask_idxs, batch_labels  = [], [], []
    for sample in batch_samples:
        batch_sentences.append(sample['prompt'])
        encoding = tokenizer(sample['prompt'], truncation=True)
        mask_idx = encoding.char_to_token(sample['mask_offset'])
        assert mask_idx is not None
        batch_mask_idxs.append(mask_idx)
        batch_labels.append(int(sample['label']))
    batch_inputs = tokenizer(
        batch_sentences,
        max_length=max_length,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    label_word_id = [neg_id, pos_id]
    return {
        'batch_inputs': batch_inputs,
        'batch_mask_idxs': batch_mask_idxs,
        'label_word_id': label_word_id,
        'labels': batch_labels
    }

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_dataloader = DataLoader(valid_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

def batched_index_select(input, dim, index):
    for i in range(1, len(input.shape)):
        if i != dim:
            index = index.unsqueeze(i)
    expanse = list(input.shape)
    expanse[0] = -1
    expanse[dim] = -1
    index = index.expand(expanse)
    return torch.gather(input, dim, index)

class BertPredictionHeadTransform(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        if isinstance(config.hidden_act, str):
            self.transform_act_fn = ACT2FN[config.hidden_act]
        else:
            self.transform_act_fn = config.hidden_act
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        hidden_states = self.dense(hidden_states)
        hidden_states = self.transform_act_fn(hidden_states)
        hidden_states = self.LayerNorm(hidden_states)
        return hidden_states

class BertLMPredictionHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.transform = BertPredictionHeadTransform(config)
        self.decoder = nn.Linear(config.hidden_size, config.vocab_size, bias=False)
        self.bias = nn.Parameter(torch.zeros(config.vocab_size))
        self.decoder.bias = self.bias

    def forward(self, hidden_states):
        hidden_states = self.transform(hidden_states)
        hidden_states = self.decoder(hidden_states)
        return hidden_states

class BertOnlyMLMHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.predictions = BertLMPredictionHead(config)

    def forward(self, sequence_output: torch.Tensor) -> torch.Tensor:
        prediction_scores = self.predictions(sequence_output)
        return prediction_scores

class BertForPrompt(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config, add_pooling_layer=False)
        self.cls = BertOnlyMLMHead(config)
        # Initialize weights and apply final processing
        self.post_init()

    def get_output_embeddings(self):
        return self.cls.predictions.decoder

    def set_output_embeddings(self, new_embeddings):
        self.cls.predictions.decoder = new_embeddings

    def forward(self, batch_inputs, batch_mask_idxs, label_word_id, labels=None):
        bert_output = self.bert(**batch_inputs)
        sequence_output = bert_output.last_hidden_state
        batch_mask_reps = batched_index_select(sequence_output, 1, batch_mask_idxs.unsqueeze(-1)).squeeze(1)
        pred_scores = self.cls(batch_mask_reps)[:, label_word_id]

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(pred_scores, labels)
        return loss, pred_scores

config = AutoConfig.from_pretrained(checkpoint)
model = BertForPrompt.from_pretrained(checkpoint, config=config).to(device)
if vtype == 'virtual':
    model.resize_token_embeddings(len(tokenizer))
    print(f"initialize embeddings of {verbalizer['pos']['token']} and {verbalizer['neg']['token']}")
    with torch.no_grad():
        pos_tokenized = tokenizer(verbalizer['pos']['description'])
        pos_tokenized_ids = tokenizer.convert_tokens_to_ids(pos_tokenized)
        neg_tokenized = tokenizer(verbalizer['neg']['description'])
        neg_tokenized_ids = tokenizer.convert_tokens_to_ids(neg_tokenized)
        new_embedding = model.bert.embeddings.word_embeddings.weight[pos_tokenized_ids].mean(axis=0)
        model.bert.embeddings.word_embeddings.weight[pos_id, :] = new_embedding.clone().detach().requires_grad_(True)
        new_embedding = model.bert.embeddings.word_embeddings.weight[neg_tokenized_ids].mean(axis=0)
        model.bert.embeddings.word_embeddings.weight[neg_id, :] = new_embedding.clone().detach().requires_grad_(True)

def to_device(batch_data):
    new_batch_data = {}
    for k, v in batch_data.items():
        if k == 'batch_inputs':
            new_batch_data[k] = {
                k_: v_.to(device) for k_, v_ in v.items()
            }
        elif k == 'label_word_id':
            new_batch_data[k] = v
        else:
            new_batch_data[k] = torch.tensor(v).to(device)
    return new_batch_data

def train_loop(dataloader, model, optimizer, lr_scheduler, epoch, total_loss):
    progress_bar = tqdm(range(len(dataloader)))
    progress_bar.set_description(f'loss: {0:>7f}')
    finish_batch_num = epoch * len(dataloader)

    model.train()
    for step, batch_data in enumerate(dataloader, start=1):
        batch_data = to_device(batch_data)
        outputs = model(**batch_data)
        loss = outputs[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
        progress_bar.set_description(f'loss: {total_loss/(finish_batch_num + step):>7f}')
        progress_bar.update(1)
    return total_loss

def test_loop(dataloader, model):
    true_labels, predictions = [], []
    model.eval()
    with torch.no_grad():
        for batch_data in tqdm(dataloader):
            true_labels += batch_data['labels']
            batch_data = to_device(batch_data)
            outputs = model(**batch_data)
            pred = outputs[1]
            predictions += pred.argmax(dim=-1).cpu().numpy().tolist()
    metrics = classification_report(true_labels, predictions, output_dict=True)
    pos_p, pos_r, pos_f1 = metrics['1']['precision'], metrics['1']['recall'], metrics['1']['f1-score']
    neg_p, neg_r, neg_f1 = metrics['0']['precision'], metrics['0']['recall'], metrics['0']['f1-score']
    macro_f1, micro_f1 = metrics['macro avg']['f1-score'], metrics['weighted avg']['f1-score']
    print(f"pos: {pos_p*100:>0.2f} / {pos_r*100:>0.2f} / {pos_f1*100:>0.2f}, neg: {neg_p*100:>0.2f} / {neg_r*100:>0.2f} / {neg_f1*100:>0.2f}")
    print(f"Macro-F1: {macro_f1*100:>0.2f} Micro-F1: {micro_f1*100:>0.2f}\n")
    return metrics

optimizer = AdamW(model.parameters(), lr=learning_rate)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_f1_score = 0.
for epoch in range(epoch_num):
    print(f"Epoch {epoch+1}/{epoch_num}\n" + 30 * "-")
    total_loss = train_loop(train_dataloader, model, optimizer, lr_scheduler, epoch, total_loss)
    valid_scores = test_loop(valid_dataloader, model)
    macro_f1, micro_f1 = valid_scores['macro avg']['f1-score'], valid_scores['weighted avg']['f1-score']
    f1_score = (macro_f1 + micro_f1) / 2
    if f1_score > best_f1_score:
        best_f1_score = f1_score
        print('saving new weights...\n')
        torch.save(
            model.state_dict(),
            f'epoch_{epoch+1}_valid_macrof1_{(macro_f1*100):0.3f}_microf1_{(micro_f1*100):0.3f}_model_weights.bin'
        )
print("Done!")


Using cuda device


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

...

  0%|          | 0/300 [00:00<?, ?it/s]

pos: 94.35 / 92.92 / 93.63, neg: 93.18 / 94.56 / 93.87
Macro-F1: 93.75 Micro-F1: 93.75

saving new weights...

Epoch 3/3
------------------------------


  0%|          | 0/2400 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

pos: 94.89 / 93.93 / 94.41, neg: 94.13 / 95.06 / 94.59
Macro-F1: 94.50 Micro-F1: 94.50

saving new weights...

Done!


# 测试模型

In [9]:
import json
# 训练完成后，加载在验证集上性能最优的模型权重，汇报其在测试集上的性能
model.load_state_dict(torch.load('epoch_3_valid_macrof1_94.498_microf1_94.500_model_weights.bin'))

model.eval()
with torch.no_grad():
    print('evaluating on test set...')
    true_labels, predictions, probs = [], [], []
    for batch_data in tqdm(test_dataloader):
        true_labels += batch_data['labels']
        batch_data = to_device(batch_data)
        outputs = model(**batch_data)
        pred = outputs[1]
        predictions += pred.argmax(dim=-1).cpu().numpy().tolist()
        probs += torch.nn.functional.softmax(pred, dim=-1)
    save_resluts = []
    for s_idx in tqdm(range(len(test_data))):
        save_resluts.append({
            "prompt": test_data[s_idx]['prompt'],
            "label": true_labels[s_idx],
            "pred": predictions[s_idx],
            "prob": {'neg': probs[s_idx][0].item(), 'pos': probs[s_idx][1].item()}
        })
    metrics = classification_report(true_labels, predictions, output_dict=True)
    pos_p, pos_r, pos_f1 = metrics['1']['precision'], metrics['1']['recall'], metrics['1']['f1-score']
    neg_p, neg_r, neg_f1 = metrics['0']['precision'], metrics['0']['recall'], metrics['0']['f1-score']
    macro_f1, micro_f1 = metrics['macro avg']['f1-score'], metrics['weighted avg']['f1-score']
    print(f"pos: {pos_p*100:>0.2f} / {pos_r*100:>0.2f} / {pos_f1*100:>0.2f}, neg: {neg_p*100:>0.2f} / {neg_r*100:>0.2f} / {neg_f1*100:>0.2f}")
    print(f"Macro-F1: {macro_f1*100:>0.2f} Micro-F1: {micro_f1*100:>0.2f}\n")
    print('saving predicted results...')
    with open('test_data_pred.json', 'wt', encoding='utf-8') as f:
        for example_result in save_resluts:
            f.write(json.dumps(example_result, ensure_ascii=False) + '\n')


evaluating on test set...


  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

pos: 96.77 / 93.75 / 95.24, neg: 93.78 / 96.79 / 95.26
Macro-F1: 95.25 Micro-F1: 95.25

saving predicted results...


可以看到，经过微调，模型在测试集上的 Macro-F1 值从 43.02 提升到 95.75，Micro-F1 值从 49.75 提升到 95.25，证明了我们对模型的微调是成功的。



In [11]:
# 打开保存预测结果的 test_data_pred.json，其中每一行对应一个样本，comment 对应评论，label 对应标注标签，pred 对应预测出的标签，prediction 对应具体预测出的概率值。
!head test_data_pred.json
!echo -e "\n"
!tail test_data_pred.json


{"prompt": "总体上来说很[MASK]。这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般", "label": 1, "pred": 0, "prob": {"neg": 0.9937229752540588, "pos": 0.006276995409280062}}
{"prompt": "总体上来说很[MASK]。怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片！开始还怀疑是不是赠送的个别现象，可是后来发现每张DVD后面都有！真不知道生产商怎么想的，我想看的是猫和老鼠，不是米老鼠！如果厂家是想赠送的话，那就全套米老鼠和唐老鸭都赠送，只在每张DVD后面添加一集算什么？？简直是画蛇添足！！", "label": 0, "pred": 0, "prob": {"neg": 0.9322289228439331, "pos": 0.0677710622549057}}
{"prompt": "总体上来说很[MASK]。还稍微重了点，可能是硬盘大的原故，还要再轻半斤就好了。其他要进一步验证。贴的几种膜气泡较多，用不了多久就要更换了，屏幕膜稍好点，但比没有要强多了。建议配赠几张膜让用用户自己贴。", "label": 0, "pred": 0, "prob": {"neg": 0.9997602105140686, "pos": 0.00023981166305020452}}
{"prompt": "总体上来说很[MASK]。交通方便；环境很好；服务态度很好 房间较小", "label": 1, "pred": 1, "prob": {"neg": 0.002282310277223587, "pos": 0.9977177381515503}}
{"prompt": "总体上来说很[MASK]。不错，作者的观点很颠覆目前中国父母的教育方式，其实古人们对于教育已经有了很系统的体系了，可是现在的父母以及祖父母们更多的娇惯纵容孩子，放眼看去自私的孩子是大多数，父母觉得自己的孩子在外面只要不吃亏就是好事，完全把古人几千年总结的教育古训抛在的九霄云外。所以推荐准妈妈们可以在等待宝宝降临的时候，好好学习一下，怎么把孩子教育成一个有爱心、有责任心、宽容、大度的人。", "label": 1, "pred": 1, "prob": {"ne

# 封装预测函数
训练模型的目的是为了能够给其他人提供服务。尤其对于不熟悉深度学习的普通开发者而言，需要的只是一个能够完成特定任务的接口。因此在大多数情况下，我们都应该将模型的预测过程封装为一个端到端 (End-to-End) 的函数：输入文本，输出结果

In [12]:
def predict(model, tokenizer, comment, verbalizer):
    prompt_data = get_prompt(comment)
    prompt = prompt_data['prompt']
    encoding = tokenizer(prompt, truncation=True)
    mask_idx = encoding.char_to_token(prompt_data['mask_offset'])
    assert mask_idx is not None
    inputs = tokenizer(
        prompt,
        max_length=max_length,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    inputs = {
        'batch_inputs': inputs,
        'batch_mask_idxs': [mask_idx],
        'label_word_id': [verbalizer['neg']['id'], verbalizer['pos']['id']]
    }
    inputs = to_device(inputs)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs[1]
        prob = torch.nn.functional.softmax(logits, dim=-1)
    pred = logits.argmax(dim=-1)[0].item()
    prob = prob[0][pred].item()
    return pred, prob


In [18]:

model.load_state_dict(torch.load('epoch_3_valid_macrof1_94.498_microf1_94.500_model_weights.bin'))

# 输出模型对测试集前 10 条数据的预测结果
for i in range(10):
    data = test_data[i]
    pred, prob = predict(model, tokenizer, data['prompt'], verbalizer)
    print(f"{data['prompt']}\nlabel: {data['label']}\tpred: {pred}\tprob: {prob}")


总体上来说很[MASK]。这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般
label: 1	pred: 0	prob: 0.989702582359314
总体上来说很[MASK]。怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片！开始还怀疑是不是赠送的个别现象，可是后来发现每张DVD后面都有！真不知道生产商怎么想的，我想看的是猫和老鼠，不是米老鼠！如果厂家是想赠送的话，那就全套米老鼠和唐老鸭都赠送，只在每张DVD后面添加一集算什么？？简直是画蛇添足！！
label: 0	pred: 0	prob: 0.8073692917823792
总体上来说很[MASK]。还稍微重了点，可能是硬盘大的原故，还要再轻半斤就好了。其他要进一步验证。贴的几种膜气泡较多，用不了多久就要更换了，屏幕膜稍好点，但比没有要强多了。建议配赠几张膜让用用户自己贴。
label: 0	pred: 0	prob: 0.9976571798324585
总体上来说很[MASK]。交通方便；环境很好；服务态度很好 房间较小
label: 1	pred: 1	prob: 0.9975438714027405
总体上来说很[MASK]。不错，作者的观点很颠覆目前中国父母的教育方式，其实古人们对于教育已经有了很系统的体系了，可是现在的父母以及祖父母们更多的娇惯纵容孩子，放眼看去自私的孩子是大多数，父母觉得自己的孩子在外面只要不吃亏就是好事，完全把古人几千年总结的教育古训抛在的九霄云外。所以推荐准妈妈们可以在等待宝宝降临的时候，好好学习一下，怎么把孩子教育成一个有爱心、有责任心、宽容、大度的人。
label: 1	pred: 1	prob: 0.9997696280479431
总体上来说很[MASK]。有了第一本书的铺垫，读第二本的时候开始进入状态。基本上第二本就围绕主角们的能力训练展开，故事的主要发生场地设置在美洲的亚马逊丛林。心里一直疑惑这和西藏有什么关系，不过大概看完全书才能知道内里的线索。其中描述了很多热带雨林中特有的神秘动植物以及一些生存技巧和常识，受益匪浅。能够想像出要写这样一部书，融合这样许多的知识，作者需要花费多少心血来搜集和整理并成文。
label: 1	pred: 1	prob: 0.9999997615814209
总体上来说很[MASK]。前台接待

https://github.com/jsksxs360/How-to-use-Transformers/tree/main/src/text_cls_prompt_senti_chnsenticorp

In [19]:
!git clone https://github.com/jsksxs360/How-to-use-Transformers.git

Cloning into 'How-to-use-Transformers'...
remote: Enumerating objects: 515, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 515 (delta 29), reused 34 (delta 18), pack-reused 452
Receiving objects: 100% (515/515), 16.25 MiB | 22.79 MiB/s, done.
Resolving deltas: 100% (217/217), done.


In [20]:
!cd How-to-use-Transformers/src/text_cls_prompt_senti_chnsenticorp && bash -x run_prompt_senti_bert.sh


+ export OUTPUT_DIR=./prompt_senti_bert_vbase_results/
+ OUTPUT_DIR=./prompt_senti_bert_vbase_results/
+ python3 run_prompt_senti_bert.py --output_dir=./prompt_senti_bert_vbase_results/ --model_type=bert --model_checkpoint=bert-base-chinese --vtype=base --train_file=../../data/ChnSentiCorp/train.txt --dev_file=../../data/ChnSentiCorp/dev.txt --test_file=../../data/ChnSentiCorp/test.txt --max_length=512 --learning_rate=1e-5 --num_train_epochs=3 --batch_size=4 --do_train --warmup_proportion=0. --seed=12
2023/12/12 04:32:49 - WARNING - Model - Using cuda device, n_gpu: 1
2023/12/12 04:32:49 - INFO - Model - loading pretrained model and tokenizer of bert ...
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
2023/12/12 04:32:51 - INFO -

In [21]:
!export OUTPUT_DIR=./prompt_senti_bert_vbase_results/ \
  && cd How-to-use-Transformers/src/text_cls_prompt_senti_chnsenticorp \
  && python run_prompt_senti_bert.py \
      --output_dir=$OUTPUT_DIR \
      --model_type=bert \
      --model_checkpoint=bert-base-chinese \
      --vtype=base \
      --train_file=../../data/ChnSentiCorp/train.txt \
      --dev_file=../../data/ChnSentiCorp/dev.txt \
      --test_file=../../data/ChnSentiCorp/test.txt \
      --max_length=512 \
      --learning_rate=1e-5 \
      --num_train_epochs=3 \
      --batch_size=4 \
      --do_test \
      --warmup_proportion=0. \
      --seed=12

2023/12/12 04:58:28 - WARNING - Model - Using cuda device, n_gpu: 1
2023/12/12 04:58:28 - INFO - Model - loading pretrained model and tokenizer of bert ...
2023/12/12 04:58:31 - INFO - Model - ***** Running testing *****
2023/12/12 04:58:31 - INFO - Model - loading weights from epoch_2_dev_macrof1_93.9096_microf1_93.9120_weights.bin...
100% 300/300 [00:15<00:00, 19.34it/s]
2023/12/12 04:58:47 - INFO - Model - POS: 97.55 / 91.78 / 94.58, NEG: 92.04 / 97.64 / 94.75
2023/12/12 04:58:47 - INFO - Model - Test: micro_F1 - 94.6640 macro_f1 - 94.6652
2023/12/12 04:58:47 - INFO - Model - loading weights from epoch_3_dev_macrof1_94.4988_microf1_94.4997_weights.bin...
100% 300/300 [00:16<00:00, 18.45it/s]
2023/12/12 04:59:04 - INFO - Model - POS: 97.10 / 93.75 / 95.40, NEG: 93.80 / 97.13 / 95.44
2023/12/12 04:59:04 - INFO - Model - Test: micro_F1 - 95.4163 macro_f1 - 95.4166
2023/12/12 04:59:04 - INFO - Model - loading weights from epoch_1_dev_macrof1_92.8328_microf1_92.8321_weights.bin...
100% 3

In [22]:
!export OUTPUT_DIR=./prompt_senti_bert_vbase_results/ \
  && cd How-to-use-Transformers/src/text_cls_prompt_senti_chnsenticorp \
  && python run_prompt_senti_bert.py \
      --output_dir=$OUTPUT_DIR \
      --model_type=bert \
      --model_checkpoint=bert-base-chinese \
      --vtype=base \
      --train_file=../../data/ChnSentiCorp/train.txt \
      --dev_file=../../data/ChnSentiCorp/dev.txt \
      --test_file=../../data/ChnSentiCorp/test.txt \
      --max_length=512 \
      --learning_rate=1e-5 \
      --num_train_epochs=3 \
      --batch_size=4 \
      --do_predict \
      --warmup_proportion=0. \
      --seed=12

2023/12/12 04:59:34 - WARNING - Model - Using cuda device, n_gpu: 1
2023/12/12 04:59:34 - INFO - Model - loading pretrained model and tokenizer of bert ...
2023/12/12 04:59:36 - INFO - Model - loading weights from epoch_2_dev_macrof1_93.9096_microf1_93.9120_weights.bin...
2023/12/12 04:59:37 - INFO - Model - predicting labels of epoch_2_dev_macrof1_93.9096_microf1_93.9120_weights.bin...
100% 1200/1200 [00:16<00:00, 72.30it/s]
2023/12/12 04:59:54 - INFO - Model - loading weights from epoch_3_dev_macrof1_94.4988_microf1_94.4997_weights.bin...
2023/12/12 04:59:54 - INFO - Model - predicting labels of epoch_3_dev_macrof1_94.4988_microf1_94.4997_weights.bin...
100% 1200/1200 [00:16<00:00, 70.88it/s]
2023/12/12 05:00:11 - INFO - Model - loading weights from epoch_1_dev_macrof1_92.8328_microf1_92.8321_weights.bin...
2023/12/12 05:00:11 - INFO - Model - predicting labels of epoch_1_dev_macrof1_92.8328_microf1_92.8321_weights.bin...
100% 1200/1200 [00:16<00:00, 73.24it/s]


In [24]:
!cd How-to-use-Transformers/src/text_cls_prompt_senti_chnsenticorp/prompt_senti_bert_vbase_results \
 && head epoch_3_dev_macrof1_94.4988_microf1_94.4997_weights.bin_test_data_pred.json


{"comment": "这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般", "label": 1, "pred": 0, "prob": 0.9963254332542419}
{"comment": "怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片！开始还怀疑是不是赠送的个别现象，可是后来发现每张DVD后面都有！真不知道生产商怎么想的，我想看的是猫和老鼠，不是米老鼠！如果厂家是想赠送的话，那就全套米老鼠和唐老鸭都赠送，只在每张DVD后面添加一集算什么？？简直是画蛇添足！！", "label": 0, "pred": 0, "prob": 0.9836694598197937}
{"comment": "还稍微重了点，可能是硬盘大的原故，还要再轻半斤就好了。其他要进一步验证。贴的几种膜气泡较多，用不了多久就要更换了，屏幕膜稍好点，但比没有要强多了。建议配赠几张膜让用用户自己贴。", "label": 0, "pred": 0, "prob": 0.9999934434890747}
{"comment": "交通方便；环境很好；服务态度很好 房间较小", "label": 1, "pred": 1, "prob": 0.9996517896652222}
{"comment": "不错，作者的观点很颠覆目前中国父母的教育方式，其实古人们对于教育已经有了很系统的体系了，可是现在的父母以及祖父母们更多的娇惯纵容孩子，放眼看去自私的孩子是大多数，父母觉得自己的孩子在外面只要不吃亏就是好事，完全把古人几千年总结的教育古训抛在的九霄云外。所以推荐准妈妈们可以在等待宝宝降临的时候，好好学习一下，怎么把孩子教育成一个有爱心、有责任心、宽容、大度的人。", "label": 1, "pred": 1, "prob": 0.9999880790710449}
{"comment": "有了第一本书的铺垫，读第二本的时候开始进入状态。基本上第二本就围绕主角们的能力训练展开，故事的主要发生场地设置在美洲的亚马逊丛林。心里一直疑惑这和西藏有什么关系，不过大概看完全书才能知道内里的线索。其中描述了很多热带雨林中特有的神秘动植物以及一些生存技巧和常识，受益匪浅。能够想像出要写这样一部书，融合这样许多的知识，作者需要花费多少心血来搜集和整理并成文。", "